# Benchmark de différent classifier 

## SVM classifier

Dans un premier temps, on va se servir d'un svm classifier et tester les performances

In [3]:
!pip install --no-cache-dir -r requirement.txt


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from skimage.feature import hog
import cv2  
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import skimage.io as io
from tqdm import tqdm
import numpy as np

IMG_PATH = "../DATA"
Data = pd.DataFrame()
UPLOAD_PATH = "Upload"
folders = os.listdir(IMG_PATH)


In [5]:
train_data = pd.DataFrame()



def load_data_images(folder_path):
    target = 0
    img_error = []
    df = pd.DataFrame()
    #crée un dictionnaire 
    img_dict = {}
    img_dict["Path"] = []
    img_dict["Target"] = []
    #for each class
    for sub_folder in tqdm(os.listdir(folder_path)):
        sub_folder_path = os.path.join(folder_path,sub_folder)
        target +=1 
        #for each image
        for filename in os.listdir(sub_folder_path):
            img_path = os.path.join(sub_folder_path, filename)
            try :
                #lecture de l'image en niveau de gris 
                img = io.imread(img_path, as_gray=True)
                #resize de l'image !Problème d'echelle
                resized_img = resize(img, (32, 32),anti_aliasing=True)  
                #Application de la fonction HOG pour extraire les features
                fd = hog(
                    resized_img,
                    orientations=8,
                    pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1)
                )
                img_dict["Path"].append(img_path)
                img_dict["Target"].append(target)
                
                #Sauvegarde dans le dictionnaire, 1 ligne = 1 image 
                for i in range(fd.shape[0]) :
                    #Si la colonne n'existe pas on la crée avant l'ajout 
                    colonne = f"FD{i}"
                    if colonne not in img_dict :
                        img_dict[colonne] = [] 
                    img_dict[colonne].append(fd[i])
                
                # print(images_list)
            except :
                img_error.append(f"{filename}, {sub_folder}")
        #passage du dictionnaire en Dataframe 
        df = pd.DataFrame(img_dict)
        
        # data = pd.concat([data,dict_df])
        # print(data)
    print("Image Error :\n" ,img_error)
    #Passage de Dataframe en CSV 
    df.to_csv("data.csv",sep=";",index=False)
        
    return df 

data = load_data_images(IMG_PATH)



  0%|          | 0/36 [00:00<?, ?it/s]c:\Users\Rafff\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 36/36 [32:54<00:00, 54.84s/it]

Image Error :
 ['Image_56.jpg, bell pepper']


In [9]:

img = io.imread("../Data/apple/Image_1.jpg", as_gray=True)
#resize de l'image !Problème d'echelle

print(img.shape)
resized_img = resize(img, (32, 32),anti_aliasing=True)  


(2216, 2218)


In [12]:
Flat_img = resized_img.flatten()
print(Flat_img[0])

1.0


In [ ]:
img = cv2.imread("../Data/apple/Image_1.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# Sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]

cv2.imshow('Original Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()